In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import sunau
from scipy import signal
from scipy.io import wavfile
import pandas as pd
import pywt
import librosa
#import python_speech_features
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import pywt # Python wavelet transform implementation
from sklearn.base import BaseEstimator, TransformerMixin # Interfaces and base classes for pipeline components
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from scipy.fft import fft, ifft
import pickle
import more_itertools
import SignalTransformers
from kymatio.sklearn import Scattering1D
import SignalTransformers
from sklearn.metrics import f1_score

In [2]:
def loadWaveToArray(audioPath):
    sample_rate, samples = wavfile.read(audioPath)
    return samples

In [3]:
def loadAudioAndTranscript(basePath, timeStamp):
    audioPath = basePath + "audio" + timeStamp + ".wav"
    transcriptPath = basePath + "transcript" + timeStamp + ".txt"
    transcript_df = pd.read_csv(transcriptPath, names = ["key_code", "press_release", "time_stamp"])
    transcript_df["rel_time_stamp"] = transcript_df["time_stamp"] - int(timeStamp)
    sample_rate, samples = wavfile.read(audioPath)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    spectrogram = np.log(spectrogram)
    transposed_spec = spectrogram.transpose()
    freq_list = list(frequencies)
    #freq_list = [str(f) for f in freq_list]
    #freq_list = [f + " Hz" for f in freq_list]
    audio_df = pd.DataFrame(transposed_spec, index = times, columns = freq_list )
    audio_df.index = times
    audio_df
    return transcript_df, audio_df, samples, sample_rate

In [4]:
def constructSpectrogramFigure(audio_df, transcript_df):
    fig, ax = plt.subplots(figsize=(16, 4))
    im = plt.pcolormesh(audio_df.index, audio_df.columns, audio_df.transpose(), shading='auto')
    nkeys = len(transcript_df)
    dy = 8000 / nkeys
    y_pos = 8000 + dy
    for index, row in transcript_df.iterrows():
        plt.text((row["rel_time_stamp"] - 200) / 1000, y_pos - index * dy, row["key_code"])
    i = 0
    while i < len(audio_df)/16000:
        plt.axvline(i)
        i += .1

In [5]:
import glob
BASE_DIR = "C:\\Users\\kbk17\\Espionage-Data\\"
paths = glob.glob(BASE_DIR + "*.wav")
paths

['C:\\Users\\kbk17\\Espionage-Data\\audio1657405825762.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405846489.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405863666.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405880674.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405897594.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405914498.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405931394.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657405948290.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993281866.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993301012.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993324432.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993344379.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993367459.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993394222.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993419855.wav',
 'C:\\Users\\kbk17\\Espionage-Data\\audio1657993438879.wav',
 'C:\\Users\\kbk17\\Espi

In [6]:
audio_arrays = [loadWaveToArray(path) for path in paths]
audio_arrays

[array([174, 243, 238, ..., 126, 127, 126], dtype=uint8),
 array([174, 243, 238, ..., 127, 127, 127], dtype=uint8),
 array([174, 243, 238, ..., 130, 130, 130], dtype=uint8),
 array([174, 243, 238, ..., 127, 127, 127], dtype=uint8),
 array([174, 243, 238, ..., 126, 127, 126], dtype=uint8),
 array([174, 243, 238, ..., 128, 128, 127], dtype=uint8),
 array([174, 243, 238, ..., 130, 130, 130], dtype=uint8),
 array([174, 243, 238, ..., 127, 127, 127], dtype=uint8),
 array([174, 243, 238, ..., 126, 126, 126], dtype=uint8),
 array([174, 243, 238, ..., 125, 126, 125], dtype=uint8),
 array([174, 243, 238, ..., 126, 126, 126], dtype=uint8),
 array([174, 243, 238, ..., 127, 126, 126], dtype=uint8),
 array([174, 243, 238, ..., 133, 133, 133], dtype=uint8),
 array([174, 243, 238, ..., 125, 125, 125], dtype=uint8),
 array([174, 243, 238, ..., 128, 128, 128], dtype=uint8),
 array([174, 243, 238, ..., 129, 129, 129], dtype=uint8),
 array([174, 243, 238, ..., 126, 126, 126], dtype=uint8),
 array([174, 2

In [7]:
audio_arrays[5]

array([174, 243, 238, ..., 128, 128, 127], dtype=uint8)

In [8]:
len(audio_arrays[5])

239524

In [9]:
audio_arrays[5].ndim

1

In [ ]:
  transformed_samples = fft(normalized_samples, n=self.n) # calculate fourier transform (complex numbers list)